In [88]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torchvision.models import resnet18

In [89]:
transform_train = transforms.Compose([
    transforms.Resize(232),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [90]:
train_dataset = torchvision.datasets.ImageFolder(root="/kaggle/input/sports-classification/train", transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

In [91]:
test_dataset = torchvision.datasets.ImageFolder(root="/kaggle/input/sports-classification/test", transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)

In [92]:
val_dataset = torchvision.datasets.ImageFolder(root="/kaggle/input/sports-classification/valid", transform=transform_test)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=2)

In [93]:
print("train size:",len(train_dataset))
print("val size:",len(val_dataset))
print("test size:",len(test_dataset))

train size: 13492
val size: 500
test size: 500


In [94]:
weight = torchvision.models.ResNet50_Weights.DEFAULT
model = resnet50(weights = weight)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [95]:
model.fc.out_features

1000

In [96]:
for param in model.parameters():
    param.requires_grad = True

In [97]:
custom_fc = nn.Sequential(
    nn.ReLU(),
    nn.Dropout(p = 0.5),
    nn.Linear(1000, 100))

model.fc = nn.Sequential(
    model.fc,
    custom_fc
)

In [98]:
# Training settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
#scheduler = lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

True


In [99]:
# Training Loop with Validation Loss and Accuracy
num_epochs = 100

for epoch in range(num_epochs):
    # Training Phase
    model.train()  # Set the model to training mode
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters
        
        running_train_loss += loss.item()  # Accumulate training loss
        
        # Track training accuracy
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()
    
    train_loss = running_train_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation Phase
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():  
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute validation loss
            running_val_loss += loss.item()
            
            # Track validation accuracy
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()
    
    val_loss = running_val_loss / len(test_loader)
    val_accuracy = 100 * correct_val / total_val

    # Print training and validation metrics
    print(f'Epoch [{epoch + 1}/{num_epochs}]')
    print(f'Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%')
    
    scheduler.step()  # Adjust learning rate according to the scheduler


Epoch [1/100]
Train Loss: 1.6002 | Train Accuracy: 58.81%
Val Loss: 0.7476 | Val Accuracy: 80.80%
Epoch [2/100]
Train Loss: 0.5905 | Train Accuracy: 82.69%
Val Loss: 0.6195 | Val Accuracy: 83.00%
Epoch [3/100]
Train Loss: 0.3836 | Train Accuracy: 88.84%
Val Loss: 0.5436 | Val Accuracy: 87.00%
Epoch [4/100]
Train Loss: 0.2828 | Train Accuracy: 91.64%
Val Loss: 0.2887 | Val Accuracy: 90.60%
Epoch [5/100]
Train Loss: 0.2306 | Train Accuracy: 93.10%
Val Loss: 0.3580 | Val Accuracy: 91.00%
Epoch [6/100]
Train Loss: 0.2114 | Train Accuracy: 93.66%
Val Loss: 0.6907 | Val Accuracy: 85.40%
Epoch [7/100]
Train Loss: 0.1927 | Train Accuracy: 94.36%
Val Loss: 0.3089 | Val Accuracy: 90.20%
Epoch [8/100]
Train Loss: 0.1567 | Train Accuracy: 94.89%
Val Loss: 0.3612 | Val Accuracy: 91.20%
Epoch [9/100]
Train Loss: 0.1315 | Train Accuracy: 96.01%
Val Loss: 0.3628 | Val Accuracy: 90.80%
Epoch [10/100]
Train Loss: 0.1176 | Train Accuracy: 96.52%
Val Loss: 0.4274 | Val Accuracy: 89.00%
Epoch [11/100]
Trai

KeyboardInterrupt: 

In [100]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    #model.train()
    return num_correct / num_samples

In [101]:
check_accuracy(test_loader,model)

tensor(0.9200, device='cuda:0')

In [104]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, dataloader, device):
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass to get predictions
            outputs = model(inputs)
            
            # Get the predicted class by taking the max output probability
            _, predicted = torch.max(outputs, 1)
            
            # Store predictions and true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
    
    # Convert lists to NumPy arrays for sklearn functions
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Classification Report
    print("Classification Report:")
    print(classification_report(all_labels, all_preds))
    
    # Confusion Matrix
    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))
    
    return all_preds, all_labels


predictions, true_labels = evaluate_model(model, test_loader, device='cuda')


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      0.80      0.89         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         5
           7       1.00      0.60      0.75         5
           8       1.00      1.00      1.00         5
           9       1.00      0.60      0.75         5
          10       1.00      1.00      1.00         5
          11       1.00      1.00      1.00         5
          12       1.00      0.60      0.75         5
          13       0.80      0.80      0.80         5
          14       1.00      1.00      1.00         5
          15       0.83      1.00      0.91         5
          16       0.62      1.00      0.77         5
    